In [3]:
import numpy as np
import matplotlib.pyplot as plt
import warnings
warnings.filterwarnings('ignore')

In [4]:
import sys
sys.path.append('../')
from Utils.models import *
from Utils.data import generateSynthetic, loadUCRDataID
from Utils.explanations import LIMESegment

### Load Simple Synthetic Test Set and Train CNN Model 

In [5]:
x_train, y_train, x_test_original, y_test_original = generateSynthetic('locality', 500, 10)

In [6]:
model = make_CNN_model(x_train.shape[1:])
trained_model, history = train_CNN_model(model, x_train, y_train, epochs=50)

2022-02-25 10:27:21.623170: I tensorflow/core/platform/cpu_feature_guard.cc:142] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.
2022-02-25 10:27:22.602893: I tensorflow/compiler/mlir/mlir_graph_optimization_pass.cc:185] None of the MLIR Optimization Passes are enabled (registered 2)


Epoch 1/50
13/13 [==============================] - 19s 935ms/step - loss: 0.6428 - sparse_categorical_accuracy: 0.8300 - val_loss: 0.6866 - val_sparse_categorical_accuracy: 0.9800
Epoch 2/50
13/13 [==============================] - 9s 643ms/step - loss: 0.4845 - sparse_categorical_accuracy: 1.0000 - val_loss: 0.6777 - val_sparse_categorical_accuracy: 0.5050
Epoch 3/50
13/13 [==============================] - 11s 865ms/step - loss: 0.3512 - sparse_categorical_accuracy: 1.0000 - val_loss: 0.7116 - val_sparse_categorical_accuracy: 0.5050
Epoch 4/50
13/13 [==============================] - 10s 702ms/step - loss: 0.2487 - sparse_categorical_accuracy: 1.0000 - val_loss: 0.8680 - val_sparse_categorical_accuracy: 0.5050
Epoch 5/50
13/13 [==============================] - 11s 862ms/step - loss: 0.1615 - sparse_categorical_accuracy: 1.0000 - val_loss: 1.2245 - val_sparse_categorical_accuracy: 0.5050
Epoch 6/50
13/13 [==============================] - 8s 657ms/step - loss: 0.1064 - sparse_catego

13/13 [==============================] - 5s 381ms/step - loss: 0.0020 - sparse_categorical_accuracy: 1.0000 - val_loss: 0.0137 - val_sparse_categorical_accuracy: 1.0000
Epoch 47/50
13/13 [==============================] - 6s 461ms/step - loss: 0.0025 - sparse_categorical_accuracy: 1.0000 - val_loss: 0.0069 - val_sparse_categorical_accuracy: 1.0000
Epoch 48/50
13/13 [==============================] - 4s 294ms/step - loss: 0.0021 - sparse_categorical_accuracy: 1.0000 - val_loss: 0.0119 - val_sparse_categorical_accuracy: 1.0000
Epoch 49/50
13/13 [==============================] - 4s 292ms/step - loss: 0.0022 - sparse_categorical_accuracy: 1.0000 - val_loss: 0.0059 - val_sparse_categorical_accuracy: 1.0000
Epoch 50/50
13/13 [==============================] - 5s 394ms/step - loss: 0.0018 - sparse_categorical_accuracy: 1.0000 - val_loss: 0.0044 - val_sparse_categorical_accuracy: 1.0000


In [7]:
test_loss, test_acc = trained_model.evaluate(x_test_original, y_test_original)

print("Test accuracy", test_acc)
print("Test loss", test_loss)

predictions = trained_model.predict(x_test_original)

1/1 [==============================] - 0s 75ms/step - loss: 0.0045 - sparse_categorical_accuracy: 1.0000
Test accuracy 1.0
Test loss 0.004484739154577255


### Generate explanations for every instance in simple synthetic test set

In [8]:
def rssi(sorted_indexes):
    overall_count = 0
    concordance_count = 0
    for i in sorted_indexes:
        for j in sorted_indexes:
            for k in range(0,(len(j))):
                if i[k] == j[k]:
                    concordance_count += 1
                overall_count += 1
    return concordance_count/overall_count 

### Evaluate RSSI Metric for Simple Synthetic Test Set 

In [9]:
test_set_rssi_dtw = []

for test_point in x_test_original:
    rankings = []
    for _ in range(10):
        dtw_explanation = LIMESegment(test_point.reshape(1000), trained_model, model_type='proba', distance='dtw', n=100, cp=6)
        rankings.append(np.asarray(dtw_explanation[0]).argsort())
    
    test_set_rssi_dtw.append(rssi(rankings))    
        

KeyboardInterrupt: 

In [ ]:
test_set_rssi_euc = []
for test_point in x_test_original:
    rankings = []
    for _ in range(10):
        euc_explanation = LIMESegment(test_point.reshape(1000), trained_model, model_type='proba', distance='euclidean', n=100, cp=6)
        rankings.append(np.asarray(euc_explanation[0]).argsort())
    
    test_set_rssi_euc.append(rssi(rankings))

In [ ]:
print(" Mean RSSI for DTW Explanations Synthetic Dataset: " + str(np.mean(test_set_rssi_dtw)) + "\n" +
      " Mean RSSI for Euclidean Explanations Synthetic Dataset: " + str(np.mean(test_set_rssi_euc)) + "\n")

### Repeat For Complex Synthetic Dataset 

In [ ]:
x_train, y_train, x_test_original, y_test_original = generateSynthetic('locality_complex',500,10)

In [ ]:
model = make_CNN_model(x_train.shape[1:])
trained_model, history = train_CNN_model(model, x_train, y_train, epochs=50)

In [ ]:
test_loss, test_acc = trained_model.evaluate(x_test_original, y_test_original)

print("Test accuracy", test_acc)
print("Test loss", test_loss)

predictions = trained_model.predict(x_test_original)

In [ ]:
test_set_rssi_dtw = []

for test_point in x_test_original:
    rankings = []
    for _ in range(10):
        dtw_explanation = LIMESegment(test_point.reshape(1000), trained_model, model_type='proba', distance='dtw', n=100, cp=5)
        rankings.append(np.asarray(dtw_explanation[0]).argsort())
    
    test_set_rssi_dtw.append(rssi(rankings))    

In [ ]:
test_set_rssi_euc = []
for test_point in x_test_original:
    rankings = []
    for _ in range(10):
        euc_explanation = LIMESegment(test_point.reshape(1000), trained_model, model_type='proba', distance='euclidean', n=100, cp=6)
        rankings.append(np.asarray(euc_explanation[0]).argsort())
    
    test_set_rssi_euc.append(rssi(rankings))

In [ ]:
print(" Mean RSSI for DTW Explanations Complex Synthetic Dataset: " + str(np.mean(test_set_rssi_dtw)) + "\n" +
      " Mean RSSI for Euclidean Explanations Complex Synthetic Dataset: " + str(np.mean(test_set_rrsi_euc)) + "\n")

### Repeat For ECG200 dataset

In [ ]:
x_train, y_train, x_test, y_test, classes = loadUCRDataID(5)

In [ ]:
x_train.shape

In [ ]:
model = make_CNN_model(x_train.shape[1:])
trained_model, history = train_CNN_model(model, x_train, y_train, batch_size=8, epochs=100)

In [ ]:
test_loss, test_acc = trained_model.evaluate(x_test, y_test)

print("Test accuracy", test_acc)
print("Test loss", test_loss)

predictions = trained_model.predict(x_test)

In [ ]:
test_set_rssi_dtw = []

for test_point in x_test[0:10]:
    rankings = []
    for _ in range(10):
        dtw_explanation = LIMESegment(test_point, trained_model, model_type='proba', distance='dtw', n=100, window_size=10, cp=6)
        rankings.append(np.asarray(dtw_explanation[0]).argsort())
    
    test_set_rssi_dtw.append(rssi(rankings))    

In [ ]:
test_set_rssi_euc = []
for test_point in x_test[0:10]:
    rankings = []
    for _ in range(10):
        euc_explanation = LIMESegment(test_point, trained_model, model_type='proba', distance='euclidean', n=100, window_size=10, cp=6)
        rankings.append(np.asarray(euc_explanation[0]).argsort())
    
    test_set_rssi_euc.append(rssi(rankings))

In [ ]:
print(" Mean RSSI for DTW Explanations ECG200 Dataset: " + str(np.mean(test_set_rssi_dtw)) + "\n" +
      " Mean RSSI for Euclidean Explanations ECG200 Dataset: " + str(np.mean(test_set_rssi_euc)) + "\n")